In [ ]:
!pip install transformers elasticsearch

import numpy as np
from transformers import AutoTokenizer, AutoModel
from elasticsearch import Elasticsearch
import torch

# 인증정보를 사용해 일래스틱서치 접속 정보 정의
es = Elasticsearch(
    ['https://hostname:port'],
    http_auth=('username', 'password'),
    verify_certs=False
)

# dense vector 필드를 위한 매핑 정의
mapping = {
    'properties': {
        'embedding': {
            'type': 'dense_vector',
            'dims': 768   # dense vector의 차원수
        }
    }
}

# 정의한 매핑으로 인덱스 생성
es.indices.create(index='chapter-2', body={'mappings': mapping})

# 색인 할 문서 데이터 세트 구성
docs = [
    {
        'title': 'Document 1',
        'text': 'This is the first document.'
    },
    {
        'title': 'Document 2',
        'text': 'This is the second document.'
    },
    {
        'title': 'Document 3',
        'text': 'This is the third document.'
    }
]

# BERT 토크나이저와 모델 로드
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

# BERT를 활용하여 각 문서의 임베딩 생성
for doc in docs:
    text = doc['text']
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        output = model(**inputs).last_hidden_state.mean(dim=1).squeeze(0).numpy()
    doc['embedding'] = output.tolist()

# 일래스틱서치에 문서 색인
for doc in docs:
    es.index(index='chapter-2', body=doc)
